In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matploblib inline

UsageError: Line magic function `%matploblib` not found.


In [7]:
array1 = np.array([0,1,2,3,4,5,6,7,8])

In [8]:
array1

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [9]:
array2 = array1*100

In [10]:
array2*array1

array([   0,  100,  400,  900, 1600, 2500, 3600, 4900, 6400])

In [19]:
array1.transpose

<function ndarray.transpose>